In [2]:
# 1. 问题陈述
# 猫狗分类

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


In [3]:
# 构建神经网络
# 输入图像大小 64*64*3，
# 激活函数relu，大于0的要，小于0的不激活
# Conv2D需要复习下了

classifier = Sequential()
# Step1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(Flatten())
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(2, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [4]:
# 读取图片数据
# Fitting the CNN to the images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)


training_set = train_datagen.flow_from_directory('dataset/train_dataset',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                classes=['Cat', 'Dog'],
                                                class_mode='categorical')


Found 19998 images belonging to 2 classes.


In [5]:
# 读取test_set
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('dataset/test_dataset',
                                           target_size=(64, 64),
                                           batch_size=32,
                                           class_mode='categorical',
                                           classes=['Cat', 'Dog'])

Found 5000 images belonging to 2 classes.


In [ ]:
# 定义一个回调函数来保存验证集上表现最好的模型
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./best_model.keras',
    monitor='val_acc',  # 监控验证集上的损失函数
    save_best_only=True,  # 仅保存在验证集上表现最好的模型
    save_weights_only=False,  # 保存整个模型（包括模型架构）
    save_freq = 'epoch',
    verbose=2 # 打印保存信息
)

classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练
classifier.fit(training_set, 
                        steps_per_epoch=80,
                        epochs=4,
                        validation_data=test_set,
                        validation_steps=2000,
                        callbacks=[checkpoint_callback])

Epoch 1/4
80/80 ━━━━━━━━━━━━━━━━━━━━ 40s 467ms/step - accuracy: 0.7047 - loss: 0.5667 - val_accuracy: 0.7026 - val_loss: 0.5577
Epoch 2/4
 2/80 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.6953 - loss: 0.5420

2024-04-20 08:19:11.092946: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 39s 494ms/step - accuracy: 0.7184 - loss: 0.5591 - val_accuracy: 0.7362 - val_loss: 0.5327
Epoch 3/4
 2/80 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6484 - loss: 0.5803

2024-04-20 08:19:50.220821: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


80/80 ━━━━━━━━━━━━━━━━━━━━ 42s 528ms/step - accuracy: 0.7135 - loss: 0.5729 - val_accuracy: 0.7496 - val_loss: 0.5096
Epoch 4/4
 2/80 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.7891 - loss: 0.5126

2024-04-20 08:20:32.023595: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


69/80 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.7318 - loss: 0.5482

In [9]:
# 使用单张图做预测

import numpy as np
from keras.preprocessing import image
# best_model =  tf.keras.models.load_model('best_model.keras')
test_image = image.load_img('dataset/single_prediction/101.jfif', target_size=(64, 64))
# test_image = image.load_img('dataset/single_prediction/2.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    print("Cat")
else:
    print("Dog")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Cat


In [76]:
# 5. 规律
# acc 如果一直下降。 val_acc一直上升。
# bias就越大。
# bias越大，就说明我正在记住我现在训练的图片，
# 我并不是在 识别 到底是猫还是狗。